In [1]:
from mdml.db import MamlDB
import pandas as pd
from pymatgen.core import Structure
import json

from maml.models import AtomSets


## Use MEGNet v1 for Ef regression

In [2]:
with open("mp_ef_100.json", "r") as f:
    data = json.load(f)
    
structs = [Structure.from_dict(i) for i in data['structure']]
ef = data['Ef']

In [3]:
from maml.describers import MEGNetSite

In [4]:
describer = MEGNetSite(level=1)

2021-11-05 12:35:25.477825: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


In [5]:
features = describer.transform(structs[:80])

2021-11-05 12:35:27.320984: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


In [6]:
model = AtomSets(describer=describer,
                 input_dim=32,  # the site features have dimenions of 32
                 is_embedding=False,
                 compile_metrics=['mae'],
                 loss='mse',
                 is_classification=False
                )

In [7]:
model.fit(features, ef[:80], epochs=10)

Epoch 1/10
1/1 [==============================] - 0s 487ms/step - loss: 3.2811 - mae: 1.7833
Epoch 2/10
1/1 [==============================] - 0s 9ms/step - loss: 2.8966 - mae: 1.6719
Epoch 3/10
1/1 [==============================] - 0s 10ms/step - loss: 2.5162 - mae: 1.5530
Epoch 4/10
1/1 [==============================] - 0s 10ms/step - loss: 2.1381 - mae: 1.4243
Epoch 5/10
1/1 [==============================] - 0s 10ms/step - loss: 1.7694 - mae: 1.2855
Epoch 6/10
1/1 [==============================] - 0s 10ms/step - loss: 1.4239 - mae: 1.1381
Epoch 7/10
1/1 [==============================] - 0s 10ms/step - loss: 1.1092 - mae: 0.9813
Epoch 8/10
1/1 [==============================] - 0s 10ms/step - loss: 0.8311 - mae: 0.8251
Epoch 9/10
1/1 [==============================] - 0s 10ms/step - loss: 0.6004 - mae: 0.6896
Epoch 10/10
1/1 [==============================] - 0s 10ms/step - loss: 0.4284 - mae: 0.5720


In [8]:
test_features = describer.transform(structs[80:])
loss, metric = model.evaluate(test_features, ef[80:], True)

2/2 [==============================] - 0s 2ms/step - loss: 0.6420 - mae: 0.7488


In [9]:
print(f"The MAE is {metric:.3f} eV/atom")

The MAE is 0.749 eV/atom


### Save and load model

Similar models can be constructed using V2 and V3 by changing `level` in the `MEGNetSite` describer.

In [10]:
model.save("Ef_V1_test")

Save entire model and load as a different model but with same parameters

In [11]:
model2 = AtomSets.from_dir("Ef_V1_test")

In [12]:
test_features = describer.transform(structs[80:])
loss, metric = model2.evaluate(test_features, ef[80:], True)

2/2 [==============================] - 0s 2ms/step - loss: 0.6420 - mae: 0.7488


In [13]:
print(f"The MAE is {metric:.3f} eV/atom")

The MAE is 0.749 eV/atom


In [17]:
model.predict_objs(structs[:3]),  model2.predict_objs(structs[:3])

(array([[2.075491 ],
        [2.0623887],
        [2.0264213]], dtype=float32),
 array([[2.075491 ],
        [2.0623887],
        [2.0264213]], dtype=float32))

## Use structural vector and MLP for regression

In [21]:
from maml.describers import MEGNetStructure
from maml.models import MLP

In [26]:
describer = MEGNetStructure(mode='final', feature_batch='pandas_concat')
model = MLP(describer=describer,
           input_dim=96,
           compile_metrics=['mae'],
           loss='mse',
           is_classification=False)

In [27]:
features = describer.transform(structs[:80])

In [28]:
model.fit(features, ef[:80], epochs=10)

Epoch 1/10
3/3 [==============================] - 0s 1ms/step - loss: 2.5584 - mae: 1.3216
Epoch 2/10
3/3 [==============================] - 0s 1ms/step - loss: 1.0640 - mae: 0.8274
Epoch 3/10
3/3 [==============================] - 0s 1ms/step - loss: 0.8410 - mae: 0.7687
Epoch 4/10
3/3 [==============================] - 0s 1ms/step - loss: 0.3368 - mae: 0.4620
Epoch 5/10
3/3 [==============================] - 0s 1ms/step - loss: 0.4552 - mae: 0.5853
Epoch 6/10
3/3 [==============================] - 0s 1ms/step - loss: 0.3155 - mae: 0.4806
Epoch 7/10
3/3 [==============================] - 0s 1ms/step - loss: 0.1186 - mae: 0.2746
Epoch 8/10
3/3 [==============================] - 0s 2ms/step - loss: 0.1795 - mae: 0.3274
Epoch 9/10
3/3 [==============================] - 0s 1ms/step - loss: 0.1663 - mae: 0.3117
Epoch 10/10
3/3 [==============================] - 0s 1ms/step - loss: 0.0845 - mae: 0.2411


In [29]:
test_features = describer.transform(structs[80:])
loss, metric = model.evaluate(test_features, ef[80:], True)

1/1 [==============================] - 0s 70ms/step - loss: 0.1682 - mae: 0.3181


In [30]:
print(f"The MAE is {metric:.3f} eV/atom")

The MAE is 0.318 eV/atom
